# 1. Import the necessary libraries for you to be able to query 4square API

In [2]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [1]:
# your code here

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [1]:
# your code here

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [3]:
def foursquare_places (venue, coordinates):
    pass

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [ ]:
# your code here

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here